# Testes de Desempenho

## K-Means

Vamos agora realizar testes de ganho de velocidade de execução, comparando o desempenho do K-Means rodando na CPU com o do K-means rodando na GPU.

Desta vez, iremos utilizar datasets bem maiores e, portanto, nada triviais — como era o caso do [*Iris* Data Set](https://archive.ics.uci.edu/ml/datasets/Iris) que foi usado anteriormente apenas como uma prova de conceito e teste de corretude.

A ideia é testar se os ganhos de desempenho ao utilizarmos uma versão paralelizada em GPU diminuem, estagnam ou aumentam junto com o aumento de instâncias ou dimensionalidade do dataset.

### Código Comum

In [26]:
import kMeans as km
import pandas as pd

import time
import os

import importlib
importlib.reload(km)

# Testing imports
print(km.kMeansCPU)
print(km.kMeansGPU)

# Valor imenso para um float, para ser usado como valor inicial na variável "slowestExecTime"
FLOAT_32_BIT_MAX = 3.4028237 * (10**38)

# Configurando o Numba para não reportar erros de baixa ocupação dos streaming multiprocessors (SMs) da GPU
# Não suprimir estes erros gera um overhead bem considerável, ocasionalmente, em algumas execuções do K-Means GPU
%set_env NUMBA_CUDA_LOW_OCCUPANCY_WARNINGS 0

<function kMeansCPU at 0x757a141b1bc0>
<function kMeansGPU at 0x757a141b1c60>
env: NUMBA_CUDA_LOW_OCCUPANCY_WARNINGS=0


### Dataset 1 (N > 1.000, D = 7, K = 2) — Rice (Cammeo and Osmancik)

Foi utilizado aqui o Dataset **[Rice (Cammeo and Osmancik)](https://archive.ics.uci.edu/dataset/545/rice+cammeo+and+osmancik)**, que reúne dados expressando características morfológicas de grãos de arroz de duas espécies, extraídas a partir de fotos destes. Temos **7 variáveis (D = 7)** e **3.810 instâncias**.

Esse dataset também contém informações de classe, definindo qual a espécie real do grão de arroz: **Cammeo** ou **Osmancik**. Portanto, haverão **2 grupos de dados (K = 2)**.

Esse conjunto de dados está presente no arquivo `Rice_Cammeo_Osmancik.arff` dentro do arquivo `rice+cammeo+and+osmancik.zip` do dataset (também disponível em download direto [neste link](https://archive.ics.uci.edu/static/public/545/rice+cammeo+and+osmancik.zip)).

#### Código

In [27]:
# Novas variáveis globais
K = 2
MAX_ITERATIONS = 5000
PLOT_RESULTS = False
DEBUG = False

COMMENT_CHAR = '%'
ALTERNATIVE_COMMENT_CHARS = ['@']

datasetFilePath = './Rice_Cammeo_Osmancik.csv'

# Processando o aqruivo .arff file e convertendo para um arquivo .csv válido (com linhas comentadas)
if not os.path.exists(datasetFilePath):
    with \
        open('./rice+cammeo+and+osmancik/Rice_Cammeo_Osmancik.arff', 'r') as file,\
        open(datasetFilePath, 'w') as fileNew:

        for line in file:
            if line[0] in ALTERNATIVE_COMMENT_CHARS:
                fileNew.write(COMMENT_CHAR + ' ' + line[1:])
            else:
                fileNew.write(line)

columnNames = ['Area', 'Perimeter', 'Major_Axis_Length', 'Minor_Axis_Length', 'Eccentricity', 'Convex_Area', 'Extent', 'Class']

# Lendo dataset do arquivo
with open(datasetFilePath, 'r') as datasetFile:
    dataset = pd.read_csv(datasetFilePath, names=columnNames, sep=',', skip_blank_lines=True, comment=COMMENT_CHAR)

dataset = dataset.drop(columns=['Class'])

print(dataset)

       Area   Perimeter  Major_Axis_Length  Minor_Axis_Length  Eccentricity   
0     15231  525.578979         229.749878          85.093788      0.928882  \
1     14656  494.311005         206.020065          91.730972      0.895405   
2     14634  501.122009         214.106781          87.768288      0.912118   
3     13176  458.342987         193.337387          87.448395      0.891861   
4     14688  507.166992         211.743378          89.312454      0.906691   
...     ...         ...                ...                ...           ...   
3805  11441  415.858002         170.486771          85.756592      0.864280   
3806  11625  421.390015         167.714798          89.462570      0.845850   
3807  12437  442.498993         183.572922          86.801979      0.881144   
3808   9882  392.296997         161.193985          78.210480      0.874406   
3809  11434  404.709991         161.079269          90.868195      0.825692   

      Convex_Area    Extent  
0           15617  0.

In [30]:
# Normalizando o dataset (normalização min-max), para que todos valores estejam no intervalo [1, 10]
datasetTreated = ((dataset - dataset.min()) / (dataset.max() - dataset.min())) * 9 + 1

print(f'##### Dataset (tratado e normalizado, intervalo [1, 10]) #####\n{datasetTreated}')

##### Dataset (tratado e normalizado, intervalo [1, 10]) #####
          Area  Perimeter  Major_Axis_Length  Minor_Axis_Length  Eccentricity   
0     7.083436   8.913085           9.110943           5.791756      8.992095  \
1     6.627970   7.426854           6.832784           7.035968      7.227819   
2     6.610544   7.750595           7.609142           6.293120      8.108617   
3     5.455642   5.717221           5.615196           6.233153      7.041041   
4     6.653318   8.037925           7.382246           6.582591      7.822599   
...        ...        ...                ...                ...           ...   
3805  4.081324   3.697823           3.421444           5.916006      5.587521   
3806  4.227073   3.960771           3.155323           6.610732      4.616211   
3807  4.870269   4.964124           4.677767           6.111975      6.476267   
3808  2.846418   2.577921           2.529299           4.501406      6.121153   
3809  4.075779   3.167936           2.518285  

In [29]:

# * ####################################
# * Rodando o K-Means CPU
# * ####################################

NUMBER_OF_RUNS = 3000

totalExecTime = 0.0
slowestExecTime = -1.0
fastestExecTime = FLOAT_32_BIT_MAX

for rep in range(1, NUMBER_OF_RUNS + 1):
    startTime = time.time()
    km.kMeansCPU(datasetTreated, k=K, maxIter=MAX_ITERATIONS, printIter=False, plotResults=PLOT_RESULTS, debug=DEBUG)
    # print(f'Results:\n \n{resultsCPU}\n ')
    elapsedTime = time.time() - startTime
    print(f'Execution time for K-Means CPU run #{rep}: {elapsedTime}')
    if elapsedTime < fastestExecTime: fastestExecTime = elapsedTime
    if elapsedTime > slowestExecTime: slowestExecTime = elapsedTime
    totalExecTime += elapsedTime
    # print(f'Average execution time for K-Means CPU until now: {totalExecTime / rep}')

print(f' \nAverage execution time for K-Means CPU: {totalExecTime / NUMBER_OF_RUNS}')
print(f'Slowest execution time for K-Means CPU: {slowestExecTime}')
print(f'Fastest execution time for K-Means CPU: {fastestExecTime}')

Execution time for K-Means CPU run #1: 0.20470285415649414
Execution time for K-Means CPU run #2: 0.1302933692932129
Execution time for K-Means CPU run #3: 0.08977460861206055
Execution time for K-Means CPU run #4: 0.0636894702911377
Execution time for K-Means CPU run #5: 0.1043698787689209
Execution time for K-Means CPU run #6: 0.06327629089355469
Execution time for K-Means CPU run #7: 0.10371994972229004
Execution time for K-Means CPU run #8: 0.06265020370483398
Execution time for K-Means CPU run #9: 0.12133097648620605
Execution time for K-Means CPU run #10: 0.0875546932220459
Execution time for K-Means CPU run #11: 0.08794856071472168
Execution time for K-Means CPU run #12: 0.07972550392150879
Execution time for K-Means CPU run #13: 0.0952448844909668
Execution time for K-Means CPU run #14: 0.0540621280670166
Execution time for K-Means CPU run #15: 0.07970857620239258
Execution time for K-Means CPU run #16: 0.07176446914672852
Execution time for K-Means CPU run #17: 0.0545113086700

KeyboardInterrupt: 

In [20]:

# * ####################################
# * Rodando o K-Means GPU
# * ####################################

NUMBER_OF_RUNS = 3000

totalExecTime = 0.0
slowestExecTime = -1.0
fastestExecTime = FLOAT_32_BIT_MAX

for rep in range(1, NUMBER_OF_RUNS + 1):
    time.sleep(0.125)
    startTime = time.time()
    km.kMeansGPU(datasetTreated, k=K, maxIter=MAX_ITERATIONS, printIter=False, plotResults=PLOT_RESULTS, debug=DEBUG)
    # print(f'Results:\n \n{resultsGPU}\n ')
    elapsedTime = time.time() - startTime
    # time.sleep(0.125)
    print(f'Execution time for K-Means GPU run #{rep}: {elapsedTime}')
    if elapsedTime < fastestExecTime: fastestExecTime = elapsedTime
    if elapsedTime > slowestExecTime: slowestExecTime = elapsedTime
    totalExecTime += elapsedTime
    # print(f'Average execution time for K-Means GPU until now: {totalExecTime / rep}')

print(f' \nAverage execution time for K-Means GPU: {totalExecTime / NUMBER_OF_RUNS}')
print(f'Slowest execution time for K-Means GPU: {slowestExecTime}')
print(f'Fastest execution time for K-Means GPU: {fastestExecTime}')

Execution time for K-Means GPU run #1: 0.016821622848510742
Execution time for K-Means GPU run #2: 0.02351522445678711
Execution time for K-Means GPU run #3: 0.029029130935668945
Execution time for K-Means GPU run #4: 0.031245946884155273
Execution time for K-Means GPU run #5: 0.031249046325683594
Execution time for K-Means GPU run #6: 0.02787637710571289
Execution time for K-Means GPU run #7: 0.020571231842041016
Execution time for K-Means GPU run #8: 0.023290395736694336
Execution time for K-Means GPU run #9: 0.02770376205444336
Execution time for K-Means GPU run #10: 0.02545785903930664
Execution time for K-Means GPU run #11: 0.025613069534301758
Execution time for K-Means GPU run #12: 0.023438692092895508
Execution time for K-Means GPU run #13: 0.022549152374267578
Execution time for K-Means GPU run #14: 0.037500619888305664
Execution time for K-Means GPU run #15: 0.031835079193115234
Execution time for K-Means GPU run #16: 0.025782346725463867
Execution time for K-Means GPU run #1

### Dataset 2 (N > 10.000, D = 8, K = 2) — HTRU2

Foi utilizado aqui o Dataset **[HTRU2 (High Time Resolution Universe 2)](https://archive.ics.uci.edu/dataset/372/htru2)**, que reúne dados a respeito de emissões de sinais de rádio de banda larga obtidos através de leituras feitas com telescópios de rádio. É um dos resultados da busca por pulsares, estrelas de neutrôn que possuem uma rotação rápida e que emitem sinais de rádio banda larga detectáveis do nosso planeta. Temos **8 variáveis (D = 8)** e **17.898 instâncias**.

Esse dataset também contém informações de classe, definindo se a leitura é **positiva** ou **negativa**, a respeito do sinal candidato de fato originar ou não de um pulsar. Portanto, haverão **2 grupos de dados (K = 2)**.

Esse conjunto de dados está presente no arquivo `HTRU_2.csv` dentro do arquivo `HTRU_2.zip` do dataset (também disponível em download direto [neste link](https://archive.ics.uci.edu/static/public/372/htru2.zip)).

#### Código

In [31]:
# Novas variáveis globais
K = 2
MAX_ITERATIONS = 5000
PLOT_RESULTS = False
DEBUG = False

datasetFilePath = './htru2/HTRU_2.csv'

columnNames = ['mean_IP', 'std_dev_IP', 'exc_kurt_IP', 'skew_IP', 'mean_DM_SNR', 'std_dev_DM_SNR', 'exc_kurt_DM_SNR', 'skew_DM_SNR', 'is_positive']

# Lendo dataset do arquivo
with open(datasetFilePath, 'r') as datasetFile:
    dataset = pd.read_csv(datasetFilePath, names=columnNames, sep=',')

dataset = dataset.drop(columns=['is_positive'])

print(dataset)

          mean_IP  std_dev_IP  exc_kurt_IP   skew_IP  mean_DM_SNR   
0      140.562500   55.683782    -0.234571 -0.699648     3.199833  \
1      102.507812   58.882430     0.465318 -0.515088     1.677258   
2      103.015625   39.341649     0.323328  1.051164     3.121237   
3      136.750000   57.178449    -0.068415 -0.636238     3.642977   
4       88.726562   40.672225     0.600866  1.123492     1.178930   
...           ...         ...          ...       ...          ...   
17893  136.429688   59.847421    -0.187846 -0.738123     1.296823   
17894  122.554688   49.485605     0.127978  0.323061    16.409699   
17895  119.335938   59.935939     0.159363 -0.743025    21.430602   
17896  114.507812   53.902400     0.201161 -0.024789     1.946488   
17897   57.062500   85.797340     1.406391  0.089520   188.306020   

       std_dev_DM_SNR  exc_kurt_DM_SNR  skew_DM_SNR  
0           19.110426         7.975532    74.242225  
1           14.860146        10.576487   127.393580  
2        

In [32]:
# Normalizando o dataset (normalização min-max), para que todos valores estejam no intervalo [1, 10]
datasetTreated = ((dataset - dataset.min()) / (dataset.max() - dataset.min())) * 9 + 1

print(f'##### Dataset (tratado e normalizado, intervalo [1, 10]) #####\n{datasetTreated}')

##### Dataset (tratado e normalizado, intervalo [1, 10]) #####
        mean_IP  std_dev_IP  exc_kurt_IP   skew_IP  mean_DM_SNR   
0      7.492075    4.759187     2.485386  1.140645     1.120440  \
1      5.658651    5.148176     3.118736  1.164410     1.059040   
2      5.683117    2.771815     2.990246  1.366092     1.117270   
3      7.308394    4.940954     2.635746  1.148810     1.138310   
4      4.994689    2.933627     3.241398  1.375405     1.038944   
...         ...         ...          ...       ...          ...   
17893  7.292961    5.265529     2.527670  1.135690     1.043698   
17894  6.624482    4.005425     2.813468  1.272336     1.653146   
17895  6.469407    5.276293     2.841869  1.135059     1.855621   
17896  6.236795    4.542553     2.879693  1.227544     1.069897   
17897  3.469156    8.421307     3.970340  1.242264     8.585104   

       std_dev_DM_SNR  exc_kurt_DM_SNR  skew_DM_SNR  
0            2.023125         3.654872     1.575009  
1            1.652719   

In [8]:

# * ####################################
# * Rodando o K-Means CPU
# * ####################################

NUMBER_OF_RUNS = 2000

totalExecTime = 0.0
slowestExecTime = -1.0
fastestExecTime = FLOAT_32_BIT_MAX

for rep in range(1, NUMBER_OF_RUNS + 1):
    startTime = time.time()
    km.kMeansCPU(datasetTreated, k=K, maxIter=MAX_ITERATIONS, printIter=False, plotResults=PLOT_RESULTS, debug=DEBUG)
    # print(f'Results:\n \n{resultsCPU}\n ')
    elapsedTime = time.time() - startTime
    print(f'Execution time for K-Means CPU run #{rep}: {elapsedTime}')
    if elapsedTime < fastestExecTime: fastestExecTime = elapsedTime
    if elapsedTime > slowestExecTime: slowestExecTime = elapsedTime
    totalExecTime += elapsedTime
    # print(f'Average execution time for K-Means CPU until now: {totalExecTime / rep}')

print(f' \nAverage execution time for K-Means CPU: {totalExecTime / NUMBER_OF_RUNS}')
print(f'Slowest execution time for K-Means CPU: {slowestExecTime}')
print(f'Fastest execution time for K-Means CPU: {fastestExecTime}')

Execution time for K-Means CPU run #1: 0.386516809463501
Execution time for K-Means CPU run #2: 0.26863765716552734
Execution time for K-Means CPU run #3: 0.43999195098876953
Execution time for K-Means CPU run #4: 0.24206805229187012
Execution time for K-Means CPU run #5: 0.3245205879211426
Execution time for K-Means CPU run #6: 0.3240029811859131
Execution time for K-Means CPU run #7: 0.4376654624938965
Execution time for K-Means CPU run #8: 0.4672253131866455
Execution time for K-Means CPU run #9: 0.32550883293151855
Execution time for K-Means CPU run #10: 0.29544854164123535
Execution time for K-Means CPU run #11: 0.4377772808074951
Execution time for K-Means CPU run #12: 0.3247861862182617
Execution time for K-Means CPU run #13: 0.4657411575317383
Execution time for K-Means CPU run #14: 0.3246302604675293
Execution time for K-Means CPU run #15: 0.43775010108947754
Execution time for K-Means CPU run #16: 0.3528883457183838
Execution time for K-Means CPU run #17: 0.3526489734649658
E

In [9]:

# * ####################################
# * Rodando o K-Means GPU
# * ####################################

NUMBER_OF_RUNS = 2000

totalExecTime = 0.0
slowestExecTime = -1.0
fastestExecTime = FLOAT_32_BIT_MAX

for rep in range(1, NUMBER_OF_RUNS + 1):
    startTime = time.time()
    km.kMeansGPU(datasetTreated, k=K, maxIter=MAX_ITERATIONS, printIter=False, plotResults=PLOT_RESULTS, debug=DEBUG)
    # print(f'Results:\n \n{resultsGPU}\n ')
    elapsedTime = time.time() - startTime
    print(f'Execution time for K-Means GPU run #{rep}: {elapsedTime}')
    if elapsedTime < fastestExecTime: fastestExecTime = elapsedTime
    if elapsedTime > slowestExecTime: slowestExecTime = elapsedTime
    totalExecTime += elapsedTime
    # print(f'Average execution time for K-Means GPU until now: {totalExecTime / rep}')

print(f' \nAverage execution time for K-Means GPU: {totalExecTime / NUMBER_OF_RUNS}')
print(f'Slowest execution time for K-Means GPU: {slowestExecTime}')
print(f'Fastest execution time for K-Means GPU: {fastestExecTime}')

Execution time for K-Means GPU run #1: 0.11395835876464844
Execution time for K-Means GPU run #2: 0.0629584789276123


/run/media/vini/Stuff/Desktop/Facul/TCC2/ufu-tcc2/.venv/lib/python3.11/site-packages/numba/cuda/dispatcher.py:539: NumbaPerformanceWarning: Grid size 28 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/run/media/vini/Stuff/Desktop/Facul/TCC2/ufu-tcc2/.venv/lib/python3.11/site-packages/numba/cuda/dispatcher.py:539: NumbaPerformanceWarning: Grid size 24 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Execution time for K-Means GPU run #3: 0.08500385284423828
Execution time for K-Means GPU run #4: 0.08057904243469238
Execution time for K-Means GPU run #5: 0.06778788566589355
Execution time for K-Means GPU run #6: 0.069122314453125
Execution time for K-Means GPU run #7: 0.07471370697021484
Execution time for K-Means GPU run #8: 0.05223512649536133
Execution time for K-Means GPU run #9: 0.07316112518310547
Execution time for K-Means GPU run #10: 0.07320189476013184
Execution time for K-Means GPU run #11: 0.07957577705383301
Execution time for K-Means GPU run #12: 0.07831740379333496
Execution time for K-Means GPU run #13: 0.06876659393310547
Execution time for K-Means GPU run #14: 0.09913802146911621
Execution time for K-Means GPU run #15: 0.07914328575134277
Execution time for K-Means GPU run #16: 0.08031511306762695
Execution time for K-Means GPU run #17: 0.0722360610961914
Execution time for K-Means GPU run #18: 0.07373785972595215
Execution time for K-Means GPU run #19: 0.06707906

### Dataset 3 (N > 100.000, D = 50, K = 2) — MiniBooNE

Foi utilizado aqui o Dataset **[MiniBooNE Particle Identification](https://archive.ics.uci.edu/dataset/199/miniboone+particle+identification)**, que reúne dados a respeito de partículas detectadas no experimento *MiniBooNE* (*Mini Booster Neutrino Experiment*), conduzido no laboratório americano *Fermilab*. Cada detecção de partícula é descrita por **50 variáveis reais (D = 50)** e há **129.596 instâncias no total**.

As primeiras 36.488 instâncias são detecções de neutrinos do elétron (sinal) e as 93.108 restantes são de neutrinos do múon (ruído de fundo). Assim, as informações de classe desse dataset estão implícitas, expressa pela ordem das instâncias no arquivo. Como temos duas classes, haverão **2 grupos de dados (K = 2)**.

Esse conjunto de dados está presente no arquivo `MiniBooNE_PID.txt` dentro do arquivo `miniboone+particle+identification.zip` do dataset (também disponível em download direto [neste link](https://archive.ics.uci.edu/static/public/199/miniboone+particle+identification.zip)).

Foi necessário, neste dataset, realizar um **pré-processamento** para **remoção de outliers**. Originalmente, há 130.064 instâncias no total (36.499 sinal e 93.565 ruído). Porém, existem 468 instâncias (11 sinal e 457 ruído) que são extremos outliers, possuindo o valor -999.0 em todas as 50 variáveis — provavelmente advindos de algum erro de detecção. A presença destes outliers causava a criação de um cluster contendo apenas estes outliers, diminuindo muito o tempo de execução do algoritmo de maneira artificial. Estes outliers tiveram que ser removidos. Note que poderíamos ter solucionado este problema com outra abordagem: aumentar K para 3, criando um cluster novo para conter apenas os outliers. Isso, no entanto, seria mais custoso computacionalmente do que a remoção das instâncias.

#### Código

In [33]:
# Novas variáveis globais
K = 2
MAX_ITERATIONS = 5000
PLOT_RESULTS = False
DEBUG = False

COMMENT_CHAR = '#'

# As primeiras 36.499 instâncias são consideradas um sinal, e o resto como ruído
N_OF_SIGNAL_LINES = 36499

datasetFilePath = './MiniBooNE_PID.csv'

# Processando o aqruivo .txt file e convertendo para um arquivo .csv válido (com a primeira linha comentada, removendo o leading whitespace, e trocando o separador de "  " ou " " para ",")
if not os.path.exists(datasetFilePath):
    with \
        open('./MiniBooNE_PID.txt', 'r') as file,\
        open(datasetFilePath, 'w') as fileNew:

        print('Processing MiniBooNE_PID.txt...\n ')

        # Removendo outliers com -999.0 de valor nas 50 variáveis. Há 468 destas instâncias
        outlierString = '''-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03'''

        index = 1
        signalInstRemoved = 0
        noiseInstRemoved = 0
        for line in file:
            if index != 1:
                lineToWrite = line.strip(' ').replace('  ', ' ').replace(' ', ',')
                if outlierString not in lineToWrite:
                    fileNew.write(lineToWrite)
                else:
                    if index - 1 <= N_OF_SIGNAL_LINES:
                        # print(f'Instance (signal) #{index - 1} removed...')
                        signalInstRemoved += 1
                    else:
                        # print(f'Instance (noise) #{index - 1} removed...')
                        noiseInstRemoved += 1
            # else:
            #     fileNew.write(COMMENT_CHAR + ' ' + line.strip(' ').replace('  ', ' '))
            index += 1

        print(f'Signal outlier instances removed = {signalInstRemoved}')
        print(f'Noise outlier instances removed = {noiseInstRemoved}\n ')

        print(f'Processed dataset saved in {datasetFilePath} with success!\n ')
else:
    print(f'Processed dataset found in {datasetFilePath}. No need for processing!\n ')

columnNames = [f'id_var_{i}' for i in range(1, 50 + 1) ]

# Lendo dataset do arquivo
with open(datasetFilePath, 'r') as datasetFile:
    dataset = pd.read_csv(datasetFile, names=columnNames, sep=',', skip_blank_lines=True)

print(dataset)

Processed dataset found in ./MiniBooNE_PID.csv. No need for processing!
 
        id_var_1  id_var_2  id_var_3  id_var_4  id_var_5  id_var_6  id_var_7   
0        2.59413  0.468803   20.6916  0.322648  0.009682  0.374393  0.803479  \
1        3.86388  0.645781   18.1375  0.233529  0.030733  0.361239  1.069740   
2        3.38584  1.197140   36.0807  0.200866  0.017341  0.260841  1.108950   
3        4.28524  0.510155  674.2010  0.281923  0.009174  0.000000  0.998822   
4        5.93662  0.832993   59.8796  0.232853  0.025066  0.233556  1.370040   
...          ...       ...       ...       ...       ...       ...       ...   
129591   4.80718  1.451020  174.6920  0.343481  0.002174  0.000000  0.747401   
129592   5.00527  1.501860  129.9270  0.273477  0.006098  0.109769  1.325370   
129593   3.10842  2.178140   56.3651  0.211850  0.000000  0.167382  1.318900   
129594   5.44560  1.845700  103.4630  0.287411  0.015929  0.107495  0.679931   
129595   4.55062  1.341740   80.0887  0.283594

In [34]:
# Normalizando o dataset (normalização min-max), para que todos valores estejam no intervalo [1, 10]
datasetTreated = ((dataset - dataset.min()) / (dataset.max() - dataset.min())) * 9 + 1

print(f'##### Dataset (tratado e normalizado, intervalo [1, 10]) #####\n{datasetTreated}')

##### Dataset (tratado e normalizado, intervalo [1, 10]) #####
        id_var_1  id_var_2  id_var_3  id_var_4  id_var_5  id_var_6  id_var_7   
0       2.368749  1.421131  1.039201  4.103207  5.452597  5.787233  2.158663  \
1       3.038712  1.603309  1.034359  2.834322  6.017928  5.619037  2.542627   
2       2.786482  2.170867  1.068374  2.369263  5.658285  4.335283  2.599170   
3       3.261035  1.463698  2.278040  3.523361  5.438966  1.000000  2.440359   
4       4.132359  1.796021  1.113489  2.824697  5.865742  3.986399  2.975677   
...          ...       ...       ...       ...       ...       ...       ...   
129591  3.536428  2.432206  1.331135  4.399829  5.250969  1.000000  2.077796   
129592  3.640947  2.484539  1.246275  3.403106  5.356339  2.403578  2.911261   
129593  2.640106  3.180688  1.106826  2.525655  5.192588  3.140255  2.901930   
129594  3.873280  2.838481  1.196108  3.601499  5.620371  2.374501  1.980500   
129595  3.401059  2.319715  1.151798  3.547153  5.192588 

In [12]:

# * ####################################
# * Rodando o K-Means CPU
# * ####################################

NUMBER_OF_RUNS = 200

totalExecTime = 0.0
slowestExecTime = -1.0
fastestExecTime = FLOAT_32_BIT_MAX

for rep in range(1, NUMBER_OF_RUNS + 1):
    startTime = time.time()
    km.kMeansCPU(datasetTreated, k=K, maxIter=MAX_ITERATIONS, printIter=False, plotResults=PLOT_RESULTS, debug=DEBUG)
    # print(f'Results:\n \n{resultsCPU}\n ')
    elapsedTime = time.time() - startTime
    print(f'Execution time for K-Means CPU run #{rep}: {elapsedTime}')
    if elapsedTime < fastestExecTime: fastestExecTime = elapsedTime
    if elapsedTime > slowestExecTime: slowestExecTime = elapsedTime
    totalExecTime += elapsedTime
    # print(f'Average execution time for K-Means CPU until now: {totalExecTime / rep}')

print(f' \nAverage execution time for K-Means CPU: {totalExecTime / NUMBER_OF_RUNS}')
print(f'Slowest execution time for K-Means CPU: {slowestExecTime}')
print(f'Fastest execution time for K-Means CPU: {fastestExecTime}')

Execution time for K-Means CPU run #1: 11.72091007232666
Execution time for K-Means CPU run #2: 7.5509233474731445
Execution time for K-Means CPU run #3: 8.744991779327393
Execution time for K-Means CPU run #4: 7.55429482460022
Execution time for K-Means CPU run #5: 8.524061679840088
Execution time for K-Means CPU run #6: 9.246991157531738
Execution time for K-Means CPU run #7: 4.846573352813721
Execution time for K-Means CPU run #8: 8.03544807434082
Execution time for K-Means CPU run #9: 9.236092805862427
Execution time for K-Means CPU run #10: 6.315953731536865
Execution time for K-Means CPU run #11: 12.202248573303223
Execution time for K-Means CPU run #12: 12.681684732437134
Execution time for K-Means CPU run #13: 7.06890082359314
Execution time for K-Means CPU run #14: 9.77237057685852
Execution time for K-Means CPU run #15: 8.992178440093994
Execution time for K-Means CPU run #16: 7.059105634689331
Execution time for K-Means CPU run #17: 10.480403900146484
Execution time for K-Me

In [13]:

# * ####################################
# * Rodando o K-Means GPU
# * ####################################

NUMBER_OF_RUNS = 200

totalExecTime = 0.0
slowestExecTime = -1.0
fastestExecTime = FLOAT_32_BIT_MAX

for rep in range(1, NUMBER_OF_RUNS + 1):
    startTime = time.time()
    km.kMeansGPU(datasetTreated, k=K, maxIter=MAX_ITERATIONS, printIter=False, plotResults=PLOT_RESULTS, debug=DEBUG)
    # print(f'Results:\n \n{resultsGPU}\n ')
    elapsedTime = time.time() - startTime
    print(f'Execution time for K-Means GPU run #{rep}: {elapsedTime}')
    if elapsedTime < fastestExecTime: fastestExecTime = elapsedTime
    if elapsedTime > slowestExecTime: slowestExecTime = elapsedTime
    totalExecTime += elapsedTime
    # print(f'Average execution time for K-Means GPU until now: {totalExecTime / rep}')

print(f' \nAverage execution time for K-Means GPU: {totalExecTime / NUMBER_OF_RUNS}')
print(f'Slowest execution time for K-Means GPU: {slowestExecTime}')
print(f'Fastest execution time for K-Means GPU: {fastestExecTime}')

Execution time for K-Means GPU run #1: 2.381702423095703
Execution time for K-Means GPU run #2: 2.1276016235351562
Execution time for K-Means GPU run #3: 2.3525049686431885
Execution time for K-Means GPU run #4: 2.1725308895111084
Execution time for K-Means GPU run #5: 2.391785144805908
Execution time for K-Means GPU run #6: 2.405285596847534
Execution time for K-Means GPU run #7: 1.6755168437957764
Execution time for K-Means GPU run #8: 1.7822277545928955
Execution time for K-Means GPU run #9: 1.2774362564086914
Execution time for K-Means GPU run #10: 2.5487823486328125
Execution time for K-Means GPU run #11: 2.189342498779297
Execution time for K-Means GPU run #12: 2.275547742843628
Execution time for K-Means GPU run #13: 2.3371646404266357
Execution time for K-Means GPU run #14: 2.0226707458496094
Execution time for K-Means GPU run #15: 3.0340769290924072
Execution time for K-Means GPU run #16: 2.183748483657837
Execution time for K-Means GPU run #17: 1.5326790809631348
Execution ti

### Dataset 4 (N > 1.000.000, D = 8) — WESAD

Foi utilizado aqui um sub-conjunto dos dados do Dataset **[WESAD (Wearable Stress and Affect Detection)](https://archive.ics.uci.edu/dataset/465/wesad+wearable+stress+and+affect+detection)**, que reúne dados, fisiológicos e de movimento, de diversos sensores presentes em aparelhos *wearables* usados por 15 pacientes diferentes em testes laboratoriais. Um aparelho foi usado no peitoral e outro no pulso dos pacientes.

Esse dataset também contém informações de classe, definindo momentos dos testes como pertencendo à três classificações de emoção do paciente: **referência**, **estresse** ou **diversão**. Portanto, haverão **3 grupos de dados (K = 3)**.

O sub-conjunto de dados utilizado foi: dados obtidos apenas através do **aparelho usado no peito** do paciente, e apenas do **paciente #4**. Utilizando este sub-conjunto, temos **8 variáveis (D = 8)** e **4.588.552 instâncias**, cada uma sendo uma leitura ao longo do tempo do teste laboratorial (leituras realizadas na frequência de 700hz).

Esse sub-conjunto de dados está presente no arquivo `S4/S4_respiban.txt` dentro do arquivo `WESAD.zip` do dataset (também disponível em download direto [neste link](https://uni-siegen.sciebo.de/s/HGdUkoNlW1Ub0Gx/download)).

#### Código

In [35]:
# Novas variáveis globais
K = 3
MAX_ITERATIONS = 5000
PLOT_RESULTS = False
DEBUG = False

datasetFilePath = './WESAD/S4/S4_respiban.txt'
columnNames = ['index', 'DI', 'ECG', 'EDA', 'EMG', 'TEMP', 'spatialX', 'spatialY', 'spatialZ', 'RESPIRATION', '_ignore_']

# Lendo dataset do arquivo
with open(datasetFilePath, 'r') as datasetFile:
    dataset = pd.read_csv(datasetFilePath, names=columnNames, sep='\t', index_col=0, skip_blank_lines=True, comment='#')

dataset = dataset.drop(columns=['DI', '_ignore_'])

print(dataset)

           ECG   EDA    EMG   TEMP  spatialX  spatialY  spatialZ  RESPIRATION
index                                                                        
0        34487  2844  32819  27563     37495     32437     31921        33292
1        34274  2869  32481  27560     37485     32433     31935        33295
2        33960  2774  32431  27557     37471     32445     31927        33293
3        33737  2767  32561  27555     37485     32433     31925        33308
4        33602  2768  32696  27562     37487     32429     31909        33300
...        ...   ...    ...    ...       ...       ...       ...          ...
4588548  33272  6470  32721  26727     37539     32597     32256        31863
4588549  33389  6467  32360  26726     37543     32583     32253        31865
4588550  33497  6456  32357  26719     37530     32598     32243        31857
4588551  33499  6450  32175  26733     37539     32585     32263        31855
4588552  33425  6445  32340  26753     37525     32595     32237

In [36]:
# Normalizando o dataset (normalização min-max), para que todos valores estejam no intervalo [1, 10]
datasetTreated = ((dataset - dataset.min()) / (dataset.max() - dataset.min())) * 9 + 1

print(f'##### Dataset (tratado e normalizado, intervalo [1, 10]) #####\n{datasetTreated}')

##### Dataset (tratado e normalizado, intervalo [1, 10]) #####
              ECG       EDA       EMG      TEMP  spatialX  spatialY  spatialZ   
index                                                                           
0        5.735295  1.508909  4.175522  8.925593  5.800000  5.203079  4.600799  \
1        5.706038  1.527215  3.927640  8.903516  5.789437  5.196481  4.607789   
2        5.662907  1.457652  3.890971  8.881439  5.774648  5.216276  4.603795   
3        5.632276  1.452526  3.986310  8.866721  5.789437  5.196481  4.602796   
4        5.613733  1.453258  4.085316  8.918234  5.791549  5.189883  4.594808   
...           ...       ...       ...       ...       ...       ...       ...   
4588548  5.568405  4.164022  4.103651  2.773508  5.846479  5.467009  4.768057   
4588549  5.584475  4.161826  3.838902  2.766149  5.850704  5.443915  4.766559   
4588550  5.599310  4.153771  3.836701  2.714636  5.836972  5.468658  4.761567   
4588551  5.599585  4.149378  3.703227  2.81766

In [16]:

# * ####################################
# * Rodando o K-Means CPU
# * ####################################

NUMBER_OF_RUNS = 50

totalExecTime = 0.0
slowestExecTime = -1.0
fastestExecTime = FLOAT_32_BIT_MAX

for rep in range(1, NUMBER_OF_RUNS + 1):
    startTime = time.time()
    km.kMeansCPU(datasetTreated, k=K, maxIter=MAX_ITERATIONS, printIter=False, plotResults=PLOT_RESULTS, debug=DEBUG)
    # print(f'Results:\n \n{resultsCPU}\n ')
    elapsedTime = time.time() - startTime
    print(f'Execution time for K-Means CPU run #{rep}: {elapsedTime}')
    if elapsedTime < fastestExecTime: fastestExecTime = elapsedTime
    if elapsedTime > slowestExecTime: slowestExecTime = elapsedTime
    totalExecTime += elapsedTime
    # print(f'Average execution time for K-Means CPU until now: {totalExecTime / rep}')

print(f' \nAverage execution time for K-Means CPU: {totalExecTime / NUMBER_OF_RUNS}')
print(f'Slowest execution time for K-Means CPU: {slowestExecTime}')
print(f'Fastest execution time for K-Means CPU: {fastestExecTime}')

Execution time for K-Means CPU run #1: 131.12973356246948
Execution time for K-Means CPU run #2: 129.3718237876892
Execution time for K-Means CPU run #3: 92.20838856697083
Execution time for K-Means CPU run #4: 173.8915410041809
Execution time for K-Means CPU run #5: 107.43795824050903
Execution time for K-Means CPU run #6: 77.74150919914246
Execution time for K-Means CPU run #7: 99.81570076942444
Execution time for K-Means CPU run #8: 129.46960282325745
Execution time for K-Means CPU run #9: 77.9063093662262
Execution time for K-Means CPU run #10: 130.74145364761353
Execution time for K-Means CPU run #11: 139.4243927001953
Execution time for K-Means CPU run #12: 147.21596956253052
Execution time for K-Means CPU run #13: 101.85348558425903
Execution time for K-Means CPU run #14: 244.90633630752563
Execution time for K-Means CPU run #15: 86.35177779197693
Execution time for K-Means CPU run #16: 109.0810821056366
Execution time for K-Means CPU run #17: 207.1127369403839
Execution time fo

In [17]:

# * ####################################
# * Rodando o K-Means GPU
# * ####################################

NUMBER_OF_RUNS = 50

totalExecTime = 0.0
slowestExecTime = -1.0
fastestExecTime = FLOAT_32_BIT_MAX

for rep in range(1, NUMBER_OF_RUNS + 1):
    startTime = time.time()
    km.kMeansGPU(datasetTreated, k=K, maxIter=MAX_ITERATIONS, printIter=False, plotResults=PLOT_RESULTS, debug=DEBUG)
    # print(f'Results:\n \n{resultsGPU}\n ')
    elapsedTime = time.time() - startTime
    print(f'Execution time for K-Means GPU run #{rep}: {elapsedTime}')
    if elapsedTime < fastestExecTime: fastestExecTime = elapsedTime
    if elapsedTime > slowestExecTime: slowestExecTime = elapsedTime
    totalExecTime += elapsedTime
    # print(f'Average execution time for K-Means GPU until now: {totalExecTime / rep}')

print(f' \nAverage execution time for K-Means GPU: {totalExecTime / NUMBER_OF_RUNS}')
print(f'Slowest execution time for K-Means GPU: {slowestExecTime}')
print(f'Fastest execution time for K-Means GPU: {fastestExecTime}')

Execution time for K-Means GPU run #1: 40.835532665252686
Execution time for K-Means GPU run #2: 20.919931173324585
Execution time for K-Means GPU run #3: 25.200390338897705
Execution time for K-Means GPU run #4: 22.40498375892639
Execution time for K-Means GPU run #5: 8.191907405853271
Execution time for K-Means GPU run #6: 47.870601654052734
Execution time for K-Means GPU run #7: 39.49761199951172
Execution time for K-Means GPU run #8: 18.16865634918213
Execution time for K-Means GPU run #9: 33.86043381690979
Execution time for K-Means GPU run #10: 20.986873626708984
Execution time for K-Means GPU run #11: 27.966493368148804
Execution time for K-Means GPU run #12: 28.05574631690979
Execution time for K-Means GPU run #13: 39.56160521507263
Execution time for K-Means GPU run #14: 36.62439489364624
Execution time for K-Means GPU run #15: 28.135660886764526
Execution time for K-Means GPU run #16: 19.71050214767456
Execution time for K-Means GPU run #17: 35.58286762237549
Execution time f

#### Resultados

> Resultados completos disponíveis no arquivo `code/examples-and-tests/speedupTestsRawResults.txt`

| |Tempo médio (50 execuções)|Speedup Médio|
|-|-|-|
|K-Means CPU|~129,81s|-|
|K-Means GPU|~27,87s|~4,65x|

### Dataset 5 (N > 10.000.000, D = 3, K = 6) — HHAR

Foi utilizado aqui um sub-conjunto dos dados do Dataset **[Heterogeneity Human Activity Recognition (HHAR)](https://archive.ics.uci.edu/dataset/344/heterogeneity+activity+recognition)**, que reúne dados de movimento do giroscópio e acelerômetro presentes em aparelhos celulares (*smartphones*) e relógios (*smartwatches*) usados por 9 usuários diferentes ao realizar diversas atividades físicas diferentes ou estando em repouso.

Esse dataset também contém informações de classe, definindo momentos dos testes como pertencendo a uma de seis atividades realizadas: **ciclismo**, **repouso (sentado)**, **repouso (em pé)**, **andar**, **subir escadas** e **descer escadas**. Portanto, haverão **6 grupos de dados (K = 6)**.

O sub-conjunto de dados utilizado foi: dados obtidos apenas através do **giroscópio do smartphone** do usuário. Utilizando este sub-conjunto, temos **3 variáveis (D = 3)** e **13.932.632 instâncias**, cada uma sendo uma leitura ao longo do tempo do experimento.

Esse sub-conjunto de dados está presente no arquivo `Activity recognition exp/Phones_gyroscope.csv` dentro do arquivo `heterogeneity+activity+recognition.zip` do dataset (também disponível em download direto [neste link](https://archive.ics.uci.edu/static/public/344/heterogeneity+activity+recognition.zip)).

#### Código

In [37]:
# Novas variáveis globais
K = 6
MAX_ITERATIONS = 4294967296
PLOT_RESULTS = False
DEBUG = False

datasetFilePath = './heterogeneity+activity+recognition/Activity recognition exp/Phones_gyroscope.csv'
columnNames = ['index', 'arrival_time', 'creation_Time', 'x', 'y', 'z', 'user', 'model', 'device', 'gt']

# Lendo dataset do arquivo
with open(datasetFilePath, 'r') as datasetFile:
    dataset = pd.read_csv(datasetFile, names=columnNames, header=0, sep=',', index_col=0)

dataset = dataset.drop(columns=['arrival_time', 'creation_Time', 'user', 'model', 'device', 'gt'])

print(dataset)

              x         y         z
index                              
0      0.013748 -0.000626 -0.023376
1      0.014816 -0.001694 -0.022308
2      0.015884 -0.001694 -0.021240
3      0.016953 -0.003830 -0.020172
4      0.015884 -0.007034 -0.020172
...         ...       ...       ...
11306 -0.046844  0.337667  0.134677
11307 -0.117598  0.221777  0.131749
11308 -0.177617  0.056115  0.095152
11309 -0.195183 -0.124429  0.063191
11310 -0.162002 -0.208846  0.043184

[13932632 rows x 3 columns]


In [38]:
# Normalizando o dataset (normalização min-max), para que todos valores estejam no intervalo [1, 10]
datasetTreated = ((dataset - dataset.min()) / (dataset.max() - dataset.min())) * 9 + 1

print(f'##### Dataset (tratado e normalizado, intervalo [1, 10]) #####\n{datasetTreated}')

##### Dataset (tratado e normalizado, intervalo [1, 10]) #####
              x         y         z
index                              
0      3.820436  4.219502  5.217352
1      3.821163  4.219072  5.218209
2      3.821890  4.219072  5.219066
3      3.822618  4.218211  5.219923
4      3.821890  4.216920  5.219923
...         ...       ...       ...
11306  3.779176  4.355790  5.344196
11307  3.730996  4.309101  5.341846
11308  3.690126  4.242361  5.312475
11309  3.678164  4.169625  5.286825
11310  3.700759  4.135616  5.270769

[13932632 rows x 3 columns]


In [20]:

# * ####################################
# * Rodando o K-Means CPU
# * ####################################

NUMBER_OF_RUNS = 5

totalExecTime = 0.0
slowestExecTime = -1.0
fastestExecTime = FLOAT_32_BIT_MAX

for rep in range(1, NUMBER_OF_RUNS + 1):
    startTime = time.time()
    km.kMeansCPU(datasetTreated, k=K, maxIter=MAX_ITERATIONS, printIter=False, plotResults=PLOT_RESULTS, debug=DEBUG)
    # print(f'Results:\n \n{resultsCPU}\n ')
    elapsedTime = time.time() - startTime
    print(f'Execution time for K-Means CPU run #{rep}: {elapsedTime}')
    if elapsedTime < fastestExecTime: fastestExecTime = elapsedTime
    if elapsedTime > slowestExecTime: slowestExecTime = elapsedTime
    totalExecTime += elapsedTime
    # print(f'Average execution time for K-Means CPU until now: {totalExecTime / rep}')

print(f' \nAverage execution time for K-Means CPU: {totalExecTime / NUMBER_OF_RUNS}')
print(f'Slowest execution time for K-Means CPU: {slowestExecTime}')
print(f'Fastest execution time for K-Means CPU: {fastestExecTime}')

Execution time for K-Means CPU run #1: 1464.724158525467
Execution time for K-Means CPU run #2: 1631.5510032176971
Execution time for K-Means CPU run #3: 1388.7622044086456
Execution time for K-Means CPU run #4: 1434.7594995498657
Execution time for K-Means CPU run #5: 1463.6720283031464
 
Average execution time for K-Means CPU: 1476.6937788009643
Slowest execution time for K-Means CPU: 1631.5510032176971
Fastest execution time for K-Means CPU: 1388.7622044086456


In [21]:

# * ####################################
# * Rodando o K-Means GPU
# * ####################################

NUMBER_OF_RUNS = 5

totalExecTime = 0.0
slowestExecTime = -1.0
fastestExecTime = FLOAT_32_BIT_MAX

for rep in range(1, NUMBER_OF_RUNS + 1):
    startTime = time.time()
    km.kMeansGPU(datasetTreated, k=K, maxIter=MAX_ITERATIONS, printIter=False, plotResults=PLOT_RESULTS, debug=DEBUG)
    # print(f'Results:\n \n{resultsGPU}\n ')
    elapsedTime = time.time() - startTime
    print(f'Execution time for K-Means GPU run #{rep}: {elapsedTime}')
    if elapsedTime < fastestExecTime: fastestExecTime = elapsedTime
    if elapsedTime > slowestExecTime: slowestExecTime = elapsedTime
    totalExecTime += elapsedTime
    # print(f'Average execution time for K-Means GPU until now: {totalExecTime / rep}')

print(f' \nAverage execution time for K-Means GPU: {totalExecTime / NUMBER_OF_RUNS}')
print(f'Slowest execution time for K-Means GPU: {slowestExecTime}')
print(f'Fastest execution time for K-Means GPU: {fastestExecTime}')

Execution time for K-Means GPU run #1: 810.9785737991333
Execution time for K-Means GPU run #2: 508.7315237522125
Execution time for K-Means GPU run #3: 465.9330599308014
Execution time for K-Means GPU run #4: 528.6637589931488
Execution time for K-Means GPU run #5: 479.31747007369995
 
Average execution time for K-Means GPU: 558.7248773097992
Slowest execution time for K-Means GPU: 810.9785737991333
Fastest execution time for K-Means GPU: 465.9330599308014
